<a href="https://colab.research.google.com/github/japarra27/acamica-dsonline26/blob/main/DS_Bit%C3%A1cora_44_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping

En este Notebook pondremos en práctica los contenidos incorporados en la Bitácora. Usaremos la librería Selenium y BeautifulSoup para hacer un breve análisis del portal BBC en Español.

Como siempre, hay dos opciones para ejecutar este notebook. Uno es hacerlo en Colab; la otra, en tu computadora. Si utilizas Colab, no debes hacer nada más que correr las celdas correspondientes. Si lo vas a correr en tu computadora, 

>1. En primer lugar, debes instalar Selenium. Te dejamos la documentación necesaria [aquí](https://selenium-python.readthedocs.io/installation.html).
>2. Instala BeautifulSoup (beautifulsoup4).

**Para ejecutar en Colab**

Este bloque de código instala Selenium y permite ejecutar la librería en Google Colab. Lo encontramos en [esta consulta](https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com) de Stack Overflow. 

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


**Para ejecutar en, por ejemplo, Firefox en tu computadora**

Ten en cuenta que si usas otro navegador debes modificar las celdas de manera apropiada.

Corre las siguientes celdas así como están. Verás que te abre una ventana de tu navegador.

Si esa ventana te molesta, descomenta las intrucciones que hay en la primera celda.

In [ ]:
from selenium import webdriver

# Para que abra navegador
wd = webdriver.Firefox()


# Para que NO abra navegador
# firefox_options = webdriver.FirefoxOptions()
# firefox_options.add_argument('--headless')
# wd = webdriver.Firefox(options = firefox_options)

Testeamos si Selenium y el Webdriver están funcionando correctamente. Traemos el html del home.

In [ ]:
# Le decimos al Webdriver (wd) que entre a la BBC
wd.get('https://www.bbc.com/mundo/')

# Con el método page_source guardamos el html en una variable que lleva ese nombre
html=wd.page_source

# Imprimimos en pantalla
html

Al parecer está funcionando bien, pero lo que obtuvimos parece difícil de abordar. 

Vamos elegir una sección y explorar sus autores. Para eso usaremos la librería BeautifulSoup.

Antes de escribir el código:
1. Entramos a la sección que queremos analizar y guardamos el link.
2. Inspeccionamos el código con el click derecho en el texto que se nombra al autor. **Recomendación**: una vez que abriste el código, si das click derecho sobre la sección del código que te interesa y eliges "edit as HTML" podrás copiar la clase, tag o texto.

In [ ]:
#descargamos la librería BeautifulSoup
from bs4 import BeautifulSoup

#guardamos el link de la seccion en una variable
seccionTech =  "https://www.bbc.com/mundo/topics/cyx5krnw38vt"

#traemos el link de la sección Economía
wd.get(seccionTech)

#creamos un objeto Soup y lo imprimimos en pantalla
html = BeautifulSoup(wd.page_source)
html

Si bien sigue siendo difícil de abordar, ya tiene otra forma.

Localizamos los autores a través del tag y la clase.

In [ ]:
#cuando inspeccionés la págian, vi que los nombres de los autores están en el tag "p" de la siguiente clase
authors = html.find_all('p', class_ = "qa-contributor-name lx-stream-post__contributor-name gel-long-primer gs-u-m0")
authors

In [ ]:
# imprimimos sólo el texto
for strong_tag in authors:
      print(strong_tag.text)

¡Genial! Al parecer funciona bien. Ahora, intentemos utilizar algunas de las ventajas que nos brinda Selenium, para obtener los autores de las otras páginas. Para esto, vamos a utilizar un `for` loop.

In [ ]:
import numpy as np

# Este array determina cuantas páginas queremos scrapear
arrayDePaginas = np.arange(1, 10)
# Lista donde guardamos el nombre de los autores
listaDeAutores = []

# Con este for extraemos el nombre de los autores de cada página y los guardamos en una variable
for x in arrayDePaginas:
    wd.get("https://www.bbc.com/mundo/topics/c7zp57yyz25t/page/" + str(x))
    page = BeautifulSoup(wd.page_source)
    for strong_tag in page.find_all('p', class_ = "qa-contributor-name lx-stream-post__contributor-name gel-long-primer gs-u-m0"):
        # Antes de guardar la información en una lista, podés hacer un print para ver si está funcionando bien
        #print(strong_tag.text)
        listaDeAutores.append(strong_tag.text)

In [ ]:
listaDeAutores

In [ ]:
import pandas as pd

# Convertimos lista to numpy array
arrayDeAutores = np.asarray(listaDeAutores)
arrayDeAutores = arrayDeAutores.reshape(-1, 1) 

# Creamos un dataframe
dfDeAutores = pd.DataFrame.from_records(arrayDeAutores, columns=['autores'])
dfDeAutores

In [ ]:
#vemos la cantidad de autores
dfDeAutores.autores.unique()

In [ ]:
#vemos cuantas notas tiene cada autor
dfDeAutores.autores.value_counts()

In [ ]:
#cerramos el browser. Si lo cerras, no vas a poder seguir explorando la web o vas a tener que ejecutar todo desde el principio.
wd.close()

Esto es todo lo que vamos a hacer vinculado al scraping en este Notebook. Te invitamos a que juegues y pruebes distintas secciones del diario, cantidad de páginas scrapeadas y si te animás, otros portales.

A continuación, vamos a realizar un breve análisis de la información que recabamos. En este caso vamos a evaluar el género de los autores.

# Analizando los datos obtenidos (opcional)

Como mencionamos recién, nuestro análisis será breve: queremos determinar el género de los autores. Para eso vamos a utilizar la librería gender-guesser.

In [ ]:
# Para instalar la librería
# !pip install gender-guesser

import gender_guesser.detector as gender

# Creamos el objeto detector
d = gender.Detector()

In [ ]:
#guardamos en una variable los primeros nombres
primerNombres = []
for i in listaDeAutores:
    nombre = i.split(' ', 1)[0]
    primerNombres.append(nombre)

In [ ]:
female = 0
male = 0
mostly_male = 0
mostly_female = 0
unknown = 0

for x in primerNombres: 
    genero = d.get_gender(x)
    if genero == "female":
        female += 1
    if genero == "male":
        male += 1
    if genero == "mostly_male":
        mostly_male +=1
    if genero == "mostly_female":
        mostly_female +=1
    if genero == "unknown":
        unknown += 1

In [ ]:
#imprimimos en pantalla
print(female)
print(male)
print(mostly_male)
print(mostly_female)
print(unknown)

Eso es todo por ahora. Hemos visto las bases de dos librerías muy útiles:

*   **Selenium WebDriver**: para navegar en la web.
*   **BeautifulSoup**: para trabajar el html que obtenemos con Selenium.

¡Además aprendiste sobre html! Un conocimiento base que siempre está bueno tener en tecnología.